In [2]:
cd lag-llama

/home/user/energygpt/lagllama/lag-llama


/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
import glob
from collections import defaultdict
from datetime import datetime
from itertools import islice

from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from tqdm.autonotebook import tqdm

import torch
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset

from gluonts.dataset.pandas import PandasDataset
import pandas as pd

from lag_llama.gluon.estimator import LagLlamaEstimator

import warnings  
warnings.filterwarnings('ignore') 

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

torch.set_warn_always(False)

/tmp/ipykernel_3820527/2271831421.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [5]:
def get_lag_llama_predictions(dataset, prediction_length, device, context_length=32, use_rope_scaling=True, num_samples=100):
    ckpt = torch.load("./checkpoints/lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we use a GPU.
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    rope_scaling_arguments = {
        "type": "linear",
        "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
    }

    estimator = LagLlamaEstimator(
        ckpt_path="./checkpoints/lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length, # Lag-Llama was trained with a context length of 32, but can work with any context length

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],
        rope_scaling=rope_scaling_arguments if use_rope_scaling else None,

        batch_size=32,
        num_parallel_samples=100,
        device=device,
    )

    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)

    return forecasts, tss

In [7]:
# Data pipelining
def get_batched_data_fn(sub_df,
    batch_size: int = 128, 
    context_len: int = 168, 
    horizon_len: int = 24):
    
    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(sub_df) - (context_len + horizon_len), horizon_len):
      num_examples += 1
      examples["inputs"].append(sub_df["y"][start:(context_end := start + context_len)].tolist())
      examples["outputs"].append(sub_df["y"][context_end:(context_end + horizon_len)].tolist())
      examples['inputs_ts'].append(sub_df.index[start:(context_end := start + context_len)])
      examples["outputs_ts"].append(sub_df.index[context_end:(context_end + horizon_len)])

    return examples


In [8]:
def forecast_building(df):

    # Set numerical columns as float32
    for col in df.columns:
        # Check if column is not of string type
        if df[col].dtype != 'object' and pd.api.types.is_string_dtype(df[col]) == False:
            df[col] = df[col].astype('float32')
    
    # Create the Pandas
    dataset = PandasDataset.from_long_dataframe(df, target="target", item_id="item_id")

    
    backtest_dataset = dataset
    prediction_length = 24  # Define your prediction length. We use 24 here since the data is of hourly frequency
    num_samples = 10 # number of samples sampled from the probability distribution for each timestep 
    device = torch.device("cuda:0") # You can switch this to CPU or other GPUs if you'd like, depending on your environment    
    
    ckpt = torch.load("./checkpoints_bdg/lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we use a GPU.

    forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples=num_samples)

    evaluator = Evaluator()
    agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))     
    
    res_all = []
    for ts, fc in zip(tss, forecasts):
        res = ts[ts.index.isin(fc.index)]
        res.columns = ['y_true']
        res.insert(1, 'y_pred', fc.median)
        res_all.append(res)
   
    res_all_df = pd.concat(res_all).sort_index()
    return res_all_df, agg_metrics, ts_metrics

In [9]:
def fine_tune_and_forecast_building(train, test):

    prediction_length = 24
    context_length = 168
    num_samples = 20
    device = "cuda:0"
    
    ckpt = torch.load("./checkpoints_bdg/lag-llama.ckpt", map_location=device)
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]
    
    estimator = LagLlamaEstimator(
            ckpt_path="./checkpoints_bdg/lag-llama.ckpt",
            prediction_length=prediction_length,
            context_length=context_length,
            nonnegative_pred_samples=True,
            aug_prob=0,
            lr=5e-4,
    
            # estimator args
            input_size=estimator_args["input_size"],
            n_layer=estimator_args["n_layer"],
            n_embd_per_head=estimator_args["n_embd_per_head"],
            n_head=estimator_args["n_head"],
            time_feat=estimator_args["time_feat"],
    
            batch_size=64,
            num_parallel_samples=num_samples,
            trainer_kwargs = {"max_epochs": 50,}, # <- lightning trainer arguments. For modification refer Lag-llama github repo
        )    
    
    # Create the Pandas
    dataset_train = PandasDataset.from_long_dataframe(train, target="target", item_id="item_id")    
    predictor = estimator.train(dataset_train, cache_data=True, shuffle_buffer_length=1000)    
  

    # Create the Pandas
    dataset_test = PandasDataset.from_long_dataframe(test, target="target", item_id="item_id")    
    forecast_it, ts_it = make_evaluation_predictions(
            dataset=dataset_test,
            predictor=predictor,
            num_samples=num_samples
        )


    forecasts = list(forecast_it)
    tss = list(ts_it)

    evaluator = Evaluator()
    agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))     
    
    res_all = []
    for ts, fc in zip(tss, forecasts):
        res = ts[ts.index.isin(fc.index)]
        res.columns = ['y_true']
        res.insert(1, 'y_pred', fc.median)
        res_all.append(res)
     
    res_all_df = pd.concat(res_all).sort_index()
    return res_all_df, agg_metrics, ts_metrics

In [10]:
def process_building(df): 
    building_name = df.columns[0]
    df.columns = ['y']
    input_data = get_batched_data_fn(df, batch_size=500)
    
    windows_all = []
    counter = 1
    for inputs_ts, inputs, outputs_ts, outputs in zip(input_data['inputs_ts'], 
                                                      input_data['inputs'], 
                                                      input_data['outputs_ts'], 
                                                      input_data['outputs']):
        
        input_df = pd.DataFrame({'timestamp': inputs_ts, 
                                 'target': inputs})
        
        output_df = pd.DataFrame({'timestamp': outputs_ts, 
                                 'target': outputs})
        combined = pd.concat([input_df, output_df], axis=0)
        combined['item_id'] = str(building_name) + '_' + str(counter)
        combined['item_id_no'] = counter
        counter += 1
        windows_all.append(combined)
        
    windows_all_df = pd.concat(windows_all)
    windows_all_df.timestamp = pd.to_datetime(windows_all_df.timestamp)
    windows_all_df.set_index('timestamp', inplace=True)

    df = windows_all_df
    # Set numerical columns as float32
    for col in df.columns:
        # Check if column is not of string type
        if df[col].dtype != 'object' and pd.api.types.is_string_dtype(df[col]) == False:
            df[col] = df[col].astype('float32')

    th = df.item_id_no.max()/2
    train = df[df.item_id_no <= th]
    test  = df[df.item_id_no > th]    
    print(datetime.now(), '#items', df.item_id_no.max(), 'split', th)
 
    
    test_res, test_agg_metrics, test_ts_metrics = forecast_building(test)
    ft_res, ft_agg_metrics, ft_ts_metrics = fine_tune_and_forecast_building(train, test)
    return test_res, test_agg_metrics, test_ts_metrics, ft_res, ft_agg_metrics, ft_ts_metrics



def process_file(filename):
    df = pd.read_csv(filename)
    df = df.set_index(['timestamp'])    
    df.index = pd.to_datetime(df.index)    
    ix = pd.date_range(start = df.index.min(), end = df.index.max(), freq = 'H')
    df = df.reindex(ix)
    df = df.ffill()

    if df.shape[1] < 2:
        return None
        
    print(datetime.now(), df.shape, flush=True)

    test_res_all = []
    test_agg_metrics_all = []
    test_ts_metrics_all = []

    finetuned_res_all = []
    finetuned_agg_metrics_all = []
    finetuned_ts_metrics_all = []
    
    i = 0
    for building_name in df.columns:
        print(datetime.now(), i, '/', len(df.columns), building_name, flush=True)
        df1 = df[[building_name]]#.head(24*200)

        test_res, test_agg_metrics, test_ts_metrics, finetuned_res, finetuned_agg_metrics, finetuned_ts_metrics = process_building(df1)
        test_res['building'] = building_name
        test_res['filename'] = filename
        test_res_all.append(test_res)
        
        test_ts_metrics.insert(0, 'building', building_name)
        test_ts_metrics.insert(0, 'filename', filename)
        test_ts_metrics = test_ts_metrics.sort_values(['forecast_start'])
        test_ts_metrics_all.append(test_ts_metrics)
        
        test_agg_metrics_df = pd.DataFrame([test_agg_metrics])
        test_agg_metrics_df.insert(0, 'building', building_name)
        test_agg_metrics_df.insert(0, 'filename', filename)
        test_agg_metrics_all.append(test_agg_metrics_df)

        finetuned_res['building'] = building_name
        finetuned_res['filename'] = filename
        finetuned_res_all.append(finetuned_res)
        
        finetuned_ts_metrics.insert(0, 'building', building_name)
        finetuned_ts_metrics.insert(0, 'filename', filename)
        finetuned_ts_metrics = finetuned_ts_metrics.sort_values(['forecast_start'])
        finetuned_ts_metrics_all.append(finetuned_ts_metrics)
        
        finetuned_agg_metrics_df = pd.DataFrame([finetuned_agg_metrics])
        finetuned_agg_metrics_df.insert(0, 'building', building_name)
        finetuned_agg_metrics_df.insert(0, 'filename', filename)
        finetuned_agg_metrics_all.append(finetuned_agg_metrics_df)        

        i += 1
        if i % 2 == 0:
            print(datetime.now(), 'Saving...')

            test_res_all_df = pd.concat(test_res_all).round(6)
            test_res_all_df = test_res_all_df.reset_index()
            test_res_all_df = test_res_all_df.rename(columns={test_res_all_df.columns[0]: "timestamp" })
            test_res_all_df.to_csv(f'../forecasts_finetuned/{dataset}/{os.path.basename(filename)}', index=False)            

            test_ts_metrics_all_df = pd.concat(test_ts_metrics_all).round(6)
            test_ts_metrics_all_df.to_csv(f'../results_finetuned/{dataset}/test_ts_metrics_{os.path.basename(filename)}', index=False)            

            test_agg_metrics_all_df = pd.concat(test_agg_metrics_all).round(6)            
            test_agg_metrics_all_df.to_csv(f'../results_finetuned/{dataset}/test_agg_metrics_{os.path.basename(filename)}', index=False)            

            finetuned_res_all_df = pd.concat(finetuned_res_all).round(6)
            finetuned_res_all_df = finetuned_res_all_df.reset_index()
            finetuned_res_all_df = finetuned_res_all_df.rename(columns={finetuned_res_all_df.columns[0]: "timestamp" })
            finetuned_res_all_df.to_csv(f'../forecasts_finetuned/{dataset}/{os.path.basename(filename)}', index=False)            

            finetuned_ts_metrics_all_df = pd.concat(finetuned_ts_metrics_all).round(6)
            finetuned_ts_metrics_all_df.to_csv(f'../results_finetuned/{dataset}/finetuned_ts_metrics_{os.path.basename(filename)}', index=False)            

            finetuned_agg_metrics_all_df = pd.concat(finetuned_agg_metrics_all).round(6)            
            finetuned_agg_metrics_all_df.to_csv(f'../results_finetuned/{dataset}/finetuned_agg_metrics_{os.path.basename(filename)}', index=False)            
    
    test_res_all_df = pd.concat(test_res_all).round(6)
    test_res_all_df = test_res_all_df.reset_index()
    test_res_all_df = test_res_all_df.rename(columns={test_res_all_df.columns[0]: "timestamp" })
    test_res_all_df.to_csv(f'../forecasts_finetuned/{dataset}/{os.path.basename(filename)}', index=False)            

    test_ts_metrics_all_df = pd.concat(test_ts_metrics_all).round(6)
    test_ts_metrics_all_df.to_csv(f'../results_finetuned/{dataset}/test_ts_metrics_{os.path.basename(filename)}', index=False)            

    test_agg_metrics_all_df = pd.concat(test_agg_metrics_all).round(6)            
    test_agg_metrics_all_df.to_csv(f'../results_finetuned/{dataset}/test_agg_metrics_{os.path.basename(filename)}', index=False)            

    finetuned_res_all_df = pd.concat(finetuned_res_all).round(6)
    finetuned_res_all_df = finetuned_res_all_df.reset_index()
    finetuned_res_all_df = finetuned_res_all_df.rename(columns={finetuned_res_all_df.columns[0]: "timestamp" })
    finetuned_res_all_df.to_csv(f'../forecasts_finetuned/{dataset}/{os.path.basename(filename)}', index=False)            

    finetuned_ts_metrics_all_df = pd.concat(finetuned_ts_metrics_all).round(6)
    finetuned_ts_metrics_all_df.to_csv(f'../results_finetuned/{dataset}/finetuned_ts_metrics_{os.path.basename(filename)}', index=False)            

    finetuned_agg_metrics_all_df = pd.concat(finetuned_agg_metrics_all).round(6)            
    finetuned_agg_metrics_all_df.to_csv(f'../results_finetuned/{dataset}/finetuned_agg_metrics_{os.path.basename(filename)}', index=False)            

    return test_res_all_df, test_ts_metrics_all_df, test_agg_metrics_all_df, finetuned_res_all_df, finetuned_ts_metrics_all_df, finetuned_agg_metrics_all_df

In [11]:
files_list = glob.glob('/home/user/BuildingsBench/BuildingsBenchData/BuildingsBench/Buildings-900K-test-csv/*.csv')   #Enter Data set location in your comptuer in CSV format
print(files_list)

dataset = 'Buildings-900K-test-50epoch'  # dataset Name to be processed
os.makedirs(f'../forecasts_finetuned/{dataset}/', exist_ok = True)
os.makedirs(f'../results_finetuned/{dataset}/', exist_ok = True)

for filename in files_list:
    print(datetime.now(), filename)
    results = process_file(filename)
 
    print('')    

['/home/user/BuildingsBench/BuildingsBenchData/BuildingsBench/Buildings-900K-test-csv/G51059305.csv']
2024-08-15 11:19:22.735552 /home/user/BuildingsBench/BuildingsBenchData/BuildingsBench/Buildings-900K-test-csv/G51059305.csv
2024-08-15 11:19:22.781498 (8760, 83)
2024-08-15 11:19:22.781918 0 / 83 4390
2024-08-15 11:19:22.960478 #items 357.0 split 178.5


Running evaluation: 179it [00:00, 2269.25it/s]
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and

Training: |                                                                                                 | …

Epoch 0, global step 50: 'train_loss' reached 4.82763 (best 4.82763), saving model to '/home/user/energygpt/lagllama/lag-llama/lightning_logs/version_10922/checkpoints/epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' reached 3.87346 (best 3.87346), saving model to '/home/user/energygpt/lagllama/lag-llama/lightning_logs/version_10922/checkpoints/epoch=1-step=100.ckpt' as top 1
Epoch 2, global step 150: 'train_loss' reached 3.74765 (best 3.74765), saving model to '/home/user/energygpt/lagllama/lag-llama/lightning_logs/version_10922/checkpoints/epoch=2-step=150.ckpt' as top 1
Epoch 3, global step 200: 'train_loss' reached 3.60593 (best 3.60593), saving model to '/home/user/energygpt/lagllama/lag-llama/lightning_logs/version_10922/checkpoints/epoch=3-step=200.ckpt' as top 1
Epoch 4, global step 250: 'train_loss' reached 3.54087 (best 3.54087), saving model to '/home/user/energygpt/lagllama/lag-llama/lightning_logs/version_10922/checkpoints/epoch=4-step=250.ckpt' as top

2024-08-15 11:20:51.331225 1 / 83 16464
2024-08-15 11:20:51.498708 #items 357.0 split 178.5


...

Epoch 0, global step 50: 'train_loss' reached 2.93140 (best 2.93140), saving model to '/home/user/energygpt/lagllama/lag-llama/lightning_logs/version_11080/checkpoints/epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' reached 2.22748 (best 2.22748), saving model to '/home/user/energygpt/lagllama/lag-llama/lightning_logs/version_11080/checkpoints/epoch=1-step=100.ckpt' as top 1
Epoch 2, global step 150: 'train_loss' reached 2.18219 (best 2.18219), saving model to '/home/user/energygpt/lagllama/lag-llama/lightning_logs/version_11080/checkpoints/epoch=2-step=150.ckpt' as top 1
Epoch 3, global step 200: 'train_loss' reached 2.13416 (best 2.13416), saving model to '/home/user/energygpt/lagllama/lag-llama/lightning_logs/version_11080/checkpoints/epoch=3-step=200.ckpt' as top 1
Epoch 4, global step 250: 'train_loss' reached 2.00265 (best 2.00265), saving model to '/home/user/energygpt/lagllama/lag-llama/lightning_logs/version_11080/checkpoints/epoch=4-step=250.ckpt' as top

2024-08-15 13:22:03.374074 Saving...
2024-08-15 13:22:07.405247 82 / 83 348278
2024-08-15 13:22:07.747006 #items 357.0 split 178.5


Running evaluation: 179it [00:00, 4740.07it/s]
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and

Training: |                                                                                                 | …

Epoch 0, global step 50: 'train_loss' reached 3.83763 (best 3.83763), saving model to '/home/user/energygpt/lagllama/lag-llama/lightning_logs/version_11082/checkpoints/epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' reached 2.84873 (best 2.84873), saving model to '/home/user/energygpt/lagllama/lag-llama/lightning_logs/version_11082/checkpoints/epoch=1-step=100.ckpt' as top 1
Epoch 2, global step 150: 'train_loss' reached 2.66314 (best 2.66314), saving model to '/home/user/energygpt/lagllama/lag-llama/lightning_logs/version_11082/checkpoints/epoch=2-step=150.ckpt' as top 1
Epoch 3, global step 200: 'train_loss' reached 2.66217 (best 2.66217), saving model to '/home/user/energygpt/lagllama/lag-llama/lightning_logs/version_11082/checkpoints/epoch=3-step=200.ckpt' as top 1
Epoch 4, global step 250: 'train_loss' reached 2.45759 (best 2.45759), saving model to '/home/user/energygpt/lagllama/lag-llama/lightning_logs/version_11082/checkpoints/epoch=4-step=250.ckpt' as top

In [12]:
assert(False)

AssertionError: 